## LSTM Multi-Head para Predicción de Acciones

Pipeline completo de entrenamiento LSTM con arquitectura multi-cabeza que predice:
- **LOG_RETURN**: retornos logarítmicos (momentum/tendencia)
- **ABS_LOG_RETURN**: retornos logarítmicos absolutos (magnitud)
- **VOLATILITY**: volatilidad rodante (régimen de mercado)

**Técnicas anti-overfitting:**
- Dropout entre capas LSTM
- Regularización L2
- Early stopping
- Batch normalization
- Gradient clipping

**División temporal:**
- Train: 2015-2021
- Validation: 2021-2023
- Test: 2024

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore')

### Configuration

In [ ]:
# Rutas
DATA_PATH = '../data/stocks/processed/'
RESULTS_PATH = '../results/lstm/'
PLOTS_PATH = '../plots/lstm/'
os.makedirs(RESULTS_PATH, exist_ok=True)
os.makedirs(PLOTS_PATH, exist_ok=True)

TICKERS = ['GOOGL', 'AAPL', 'AMZN', 'META', 'MSFT', 'NVDA', 'TSLA']

# Hiperparámetros del modelo
SEQUENCE_LENGTH = 20
HIDDEN_SIZE = 64
NUM_LAYERS = 2
DROPOUT = 0.5
L2_REG = 1e-3

# Hiperparámetros de entrenamiento
BATCH_SIZE = 128
LEARNING_RATE = 0.0005
EPOCHS = 200
PATIENCE = 25

# Pesos para pérdida ponderada (para embeddings balanceados)
TARGET_WEIGHTS = {
    'LOG_RETURN': 3.0,
    'ABS_LOG_RETURN': 2.0,
    'VOLATILITY': 1.0
}

# Divisiones temporales
TRAIN_START = 2015
TRAIN_END = 2021
VAL_START = 2021
VAL_END = 2023
TEST_YEAR = 2024

# Variables objetivo
TARGETS = ['LOG_RETURN', 'ABS_LOG_RETURN', 'VOLATILITY']

# Características a usar
FEATURE_COLS = [
    'Close', 'High', 'Low', 'Open', 'Volume',
    'SMA_10', 'SMA_20', 'SMA_30',
    'UPPER_BAND', 'MIDDLE_BAND', 'LOWER_BAND',
    'MACD', 'MACD_SIGNAL', 'MACD_HIST',
    'RSI_14',
    'STOCH_K', 'STOCH_D',
    'WILLIAMS_R',
    'LOG_RETURN_HIGH', 'LOG_RETURN_LOW', 'LOG_RETURN_OPEN', 'LOG_RETURN_CLOSE',
    'REALIZED_VOL', 'PARKINSON_VOL', 'GARMAN_KLASS_VOL', 'ROGERS_SATCHELL_VOL',
    'VWAP',
    'DAY_OF_WEEK_SIN', 'DAY_OF_WEEK_COS',
    'MONTH_SIN', 'MONTH_COS',
    'DAY_OF_MONTH_SIN', 'DAY_OF_MONTH_COS',
    'QUARTER_SIN', 'QUARTER_COS'
]

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

### Carga y Preparación de Datos

In [ ]:
def load_and_split_data(ticker):
    """Carga datos y divide por rangos temporales"""
    print(f"\n{'='*80}")
    print(f"Loading {ticker}...")
    print('='*80)
    
    filepath = os.path.join(DATA_PATH, f"{ticker}_data_processed.parquet")
    df = pd.read_parquet(filepath)
    
    # Asegurar columna Date
    if 'Date' not in df.columns and df.index.name == 'Date':
        df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').reset_index(drop=True)
    
    # Añadir columna de año
    df['Year'] = df['Date'].dt.year
    
    # Verificar columnas requeridas
    missing_features = [f for f in FEATURE_COLS if f not in df.columns]
    missing_targets = [t for t in TARGETS if t not in df.columns]
    
    if missing_features:
        print(f"  Missing features: {missing_features[:5]}...")
    if missing_targets:
        print(f"  Missing targets: {missing_targets}")
        return None, None, None, None, None, None
    
    # Seleccionar solo características disponibles
    available_features = [f for f in FEATURE_COLS if f in df.columns]
    
    # Divisiones temporales
    train_df = df[(df['Year'] >= TRAIN_START) & (df['Year'] <= TRAIN_END)].reset_index(drop=True)
    val_df = df[(df['Year'] >= VAL_START) & (df['Year'] <= VAL_END)].reset_index(drop=True)
    test_df = df[df['Year'] == TEST_YEAR].reset_index(drop=True)
    
    print(f"  Train: {len(train_df)} samples ({TRAIN_START}-{TRAIN_END})")
    print(f"  Val:   {len(val_df)} samples ({VAL_START}-{VAL_END})")
    print(f"  Test:  {len(test_df)} samples ({TEST_YEAR})")
    print(f"  Features: {len(available_features)}")
    
    return train_df, val_df, test_df, available_features, df['Date'], df


def create_sequences(data, features, targets, seq_length):
    """Crea secuencias para entrada LSTM"""
    X, y = [], []
    
    for i in range(len(data) - seq_length):
        X.append(data[features].iloc[i:i+seq_length].values)
        y.append(data[targets].iloc[i+seq_length].values)
    
    return np.array(X), np.array(y)


class StockDataset(Dataset):
    """Dataset de PyTorch para secuencias de acciones"""
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### Arquitectura del Modelo LSTM Multi-Head

In [ ]:
class StockLSTMMultiHead(nn.Module):
    """
    Modelo LSTM con arquitectura multi-cabeza para generación de embeddings
    
    Arquitectura:
      [Input] → [LSTM Encoder] → [EMBEDDING] → [3 cabezas especializadas]
      
    El encoder aprende una representación compartida (embedding) que captura:
      - Momentum/tendencia (LOG_RETURN)
      - Magnitud (ABS_LOG_RETURN)
      - Régimen de volatilidad (VOLATILITY)
    """
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.3):
        super(StockLSTMMultiHead, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        
        # ===== ENCODER COMPARTIDO =====
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Batch normalization
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # ===== CABEZAS ESPECIALIZADAS =====
        # Cabeza 1: LOG_RETURN (momentum/tendencia)
        self.head1_fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.head1_fc2 = nn.Linear(hidden_size // 2, 1)
        
        # Cabeza 2: ABS_LOG_RETURN (magnitud)
        self.head2_fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.head2_fc2 = nn.Linear(hidden_size // 2, 1)
        
        # Cabeza 3: VOLATILITY (régimen)
        self.head3_fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.head3_fc2 = nn.Linear(hidden_size // 2, 1)
        
        self.relu = nn.ReLU()
    
    def forward(self, x, return_embedding=False):
        """Forward pass con opción de retornar embedding"""
        # LSTM encoder
        lstm_out, _ = self.lstm(x)
        
        # Tomar última salida como embedding
        embedding = lstm_out[:, -1, :]
        
        # Batch norm + dropout
        embedding_norm = self.batch_norm(embedding)
        embedding_dropped = self.dropout(embedding_norm)
        
        # Pasar por cada cabeza especializada
        h1 = self.relu(self.head1_fc1(embedding_dropped))
        out1 = self.head1_fc2(h1)
        
        h2 = self.relu(self.head2_fc1(embedding_dropped))
        out2 = self.head2_fc2(h2)
        
        h3 = self.relu(self.head3_fc1(embedding_dropped))
        out3 = self.head3_fc2(h3)
        
        # Concatenar salidas
        outputs = torch.cat([out1, out2, out3], dim=1)
        
        if return_embedding:
            return outputs, embedding
        return outputs
    
    def get_embedding(self, x):
        """Extraer embedding sin calcular predicciones"""
        with torch.no_grad():
            lstm_out, _ = self.lstm(x)
            embedding = lstm_out[:, -1, :]
            return embedding

### Función de Pérdida Ponderada

In [ ]:
class WeightedMSELoss(nn.Module):
    """
    Pérdida MSE ponderada para aprendizaje multi-tarea
    
    Aplica diferentes pesos a cada objetivo para balancear su contribución
    a la pérdida total. Crucial para el aprendizaje de embeddings.
    """
    def __init__(self, weights):
        super(WeightedMSELoss, self).__init__()
        self.weights = torch.tensor(list(weights.values()), dtype=torch.float32)
        
    def forward(self, predictions, targets):
        # Mover pesos al mismo dispositivo
        if self.weights.device != predictions.device:
            self.weights = self.weights.to(predictions.device)
        
        # Calcular MSE por objetivo
        mse_per_target = torch.mean((predictions - targets) ** 2, dim=0)
        
        # Aplicar pesos
        weighted_loss = torch.sum(self.weights * mse_per_target)
        
        return weighted_loss

### Función de Entrenamiento

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, patience):
    """Entrena con early stopping y learning rate scheduling"""
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []
    
    # Scheduler de learning rate
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, min_lr=1e-6
    )
    
    print("\n" + "="*80)
    print("Training started...")
    print("="*80)
    print(f"  Initial LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validación
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(DEVICE)
                y_batch = y_batch.to(DEVICE)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        
        # Actualizar learning rate
        scheduler.step(val_loss)
        
        # Imprimir progreso
        if (epoch + 1) % 10 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | LR: {current_lr:.6f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"\n⏹️  Early stopping at epoch {epoch+1}")
            print(f"  Best validation loss: {best_val_loss:.6f}")
            model.load_state_dict(best_model_state)
            break
    
    return model, train_losses, val_losses

### Funciones de Evaluación

In [ ]:
def evaluate_model(model, loader, dataset_name):
    """Evalúa modelo y calcula métricas"""
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(DEVICE)
            outputs = model(X_batch)
            all_preds.append(outputs.cpu().numpy())
            all_targets.append(y_batch.numpy())
    
    preds = np.vstack(all_preds)
    targets = np.vstack(all_targets)
    
    # Calcular métricas para cada objetivo
    metrics = {}
    for i, target_name in enumerate(TARGETS):
        mse = mean_squared_error(targets[:, i], preds[:, i])
        mae = mean_absolute_error(targets[:, i], preds[:, i])
        r2 = r2_score(targets[:, i], preds[:, i])
        
        metrics[target_name] = {
            'MSE': mse,
            'RMSE': np.sqrt(mse),
            'MAE': mae,
            'R2': r2
        }
    
    # Imprimir resultados
    print(f"\n📊 {dataset_name} Results:")
    print("-" * 80)
    for target_name, target_metrics in metrics.items():
        print(f"  {target_name}:")
        for metric_name, value in target_metrics.items():
            print(f"    {metric_name}: {value:.6f}")
    
    return metrics, preds, targets


def plot_predictions(preds, targets, ticker, dataset_name, dates_subset=None):
    """Grafica predicciones vs reales para cada objetivo"""
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    for i, target_name in enumerate(TARGETS):
        ax = axes[i]
        x = dates_subset if dates_subset is not None else range(len(targets))
        
        ax.plot(x, targets[:, i], label='Actual', alpha=0.7)
        ax.plot(x, preds[:, i], label='Predicted', alpha=0.7)
        ax.set_title(f'{ticker} - {target_name} ({dataset_name})')
        ax.set_ylabel(target_name)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Time' if dates_subset is None else 'Date')
    fig.tight_layout()
    
    plot_path = os.path.join(PLOTS_PATH, f"{ticker}_{dataset_name}_predictions.png")
    fig.savefig(plot_path, dpi=150, bbox_inches='tight')
    print(f"  📈 Saved plot: {plot_path}")
    plt.close(fig)

### Extracción de Embeddings

In [ ]:
def extract_embeddings(model, dataloader, device=DEVICE):
    """
    Extrae embeddings del modelo entrenado
    
    Returns:
        embeddings: array [num_samples, hidden_size]
        targets: array [num_samples, num_targets]
    """
    model.eval()
    all_embeddings = []
    all_targets = []
    
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch = X_batch.to(device)
            embeddings = model.get_embedding(X_batch)
            all_embeddings.append(embeddings.cpu().numpy())
            all_targets.append(y_batch.numpy())
    
    embeddings = np.vstack(all_embeddings)
    targets = np.vstack(all_targets)
    
    return embeddings, targets

### Pipeline Principal por Ticker

In [ ]:
def run_lstm_for_ticker(ticker):
    """Pipeline completo para un ticker con normalización de targets"""
    print(f"\n{'#'*80}")
    print(f"# Processing {ticker}")
    print('#'*80)
    
    # Cargar datos
    train_df, val_df, test_df, features, dates, full_df = load_and_split_data(ticker)
    if train_df is None:
        print(f"Skipping {ticker} due to missing data")
        return None
    
    # Escalar features
    print("\nScaling features...")
    feature_scaler = RobustScaler()
    train_df[features] = feature_scaler.fit_transform(train_df[features])
    val_df[features] = feature_scaler.transform(val_df[features])
    test_df[features] = feature_scaler.transform(test_df[features])
    
    # Escalar targets individualmente
    print("Scaling targets individually...")
    target_scalers = {}
    for target in TARGETS:
        scaler = RobustScaler()
        train_df[[target]] = scaler.fit_transform(train_df[[target]])
        val_df[[target]] = scaler.transform(val_df[[target]])
        test_df[[target]] = scaler.transform(test_df[[target]])
        target_scalers[target] = scaler
        print(f"  {target}: median={scaler.center_[0]:.6f}, scale={scaler.scale_[0]:.6f}")
    
    # Crear secuencias
    print("\nCreating sequences...")
    X_train, y_train = create_sequences(train_df, features, TARGETS, SEQUENCE_LENGTH)
    X_val, y_val = create_sequences(val_df, features, TARGETS, SEQUENCE_LENGTH)
    X_test, y_test = create_sequences(test_df, features, TARGETS, SEQUENCE_LENGTH)
    
    print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"  X_val:   {X_val.shape}, y_val:   {y_val.shape}")
    print(f"  X_test:  {X_test.shape}, y_test:  {y_test.shape}")
    
    # Crear DataLoaders
    train_dataset = StockDataset(X_train, y_train)
    val_dataset = StockDataset(X_val, y_val)
    test_dataset = StockDataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    # Inicializar modelo
    input_size = len(features)
    output_size = len(TARGETS)
    
    model = StockLSTMMultiHead(
        input_size=input_size,
        hidden_size=HIDDEN_SIZE,
        num_layers=NUM_LAYERS,
        output_size=output_size,
        dropout=DROPOUT
    ).to(DEVICE)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\nModel: {total_params:,} parameters")
    
    # Loss y optimizer
    criterion = WeightedMSELoss(TARGET_WEIGHTS)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)
    
    print(f"\nLoss weights: {TARGET_WEIGHTS}")
    
    # Entrenar
    model, train_losses, val_losses = train_model(
        model, train_loader, val_loader, criterion, optimizer, EPOCHS, PATIENCE
    )
    
    # Evaluar
    train_metrics, train_preds, train_targets = evaluate_model(model, train_loader, "TRAIN")
    val_metrics, val_preds, val_targets = evaluate_model(model, val_loader, "VALIDATION")
    test_metrics, test_preds, test_targets = evaluate_model(model, test_loader, "TEST")
    
    # Desnormalizar predicciones
    print("\Denormalizing predictions...")
    for i, target in enumerate(TARGETS):
        scaler = target_scalers[target]
        train_preds[:, i] = scaler.inverse_transform(train_preds[:, i].reshape(-1, 1)).flatten()
        train_targets[:, i] = scaler.inverse_transform(train_targets[:, i].reshape(-1, 1)).flatten()
        
        val_preds[:, i] = scaler.inverse_transform(val_preds[:, i].reshape(-1, 1)).flatten()
        val_targets[:, i] = scaler.inverse_transform(val_targets[:, i].reshape(-1, 1)).flatten()
        
        test_preds[:, i] = scaler.inverse_transform(test_preds[:, i].reshape(-1, 1)).flatten()
        test_targets[:, i] = scaler.inverse_transform(test_targets[:, i].reshape(-1, 1)).flatten()
    
    # Recalcular métricas
    print("\nFINAL METRICS (Denormalized):")
    train_metrics_final = {}
    val_metrics_final = {}
    test_metrics_final = {}
    
    for i, target in enumerate(TARGETS):
        for metrics_dict, preds, targets, name in [
            (train_metrics_final, train_preds, train_targets, "TRAIN"),
            (val_metrics_final, val_preds, val_targets, "VALIDATION"),
            (test_metrics_final, test_preds, test_targets, "TEST")
        ]:
            mse = mean_squared_error(targets[:, i], preds[:, i])
            mae = mean_absolute_error(targets[:, i], preds[:, i])
            r2 = r2_score(targets[:, i], preds[:, i])
            
            metrics_dict[target] = {
                'MSE': mse,
                'RMSE': np.sqrt(mse),
                'MAE': mae,
                'R2': r2
            }
    
    # Imprimir métricas finales
    for name, metrics_dict in [("TRAIN", train_metrics_final), ("VALIDATION", val_metrics_final), ("TEST", test_metrics_final)]:
        print(f"\n  {name}:")
        for target, target_metrics in metrics_dict.items():
            print(f"    {target}: R²={target_metrics['R2']:.4f}, RMSE={target_metrics['RMSE']:.6f}")
    
    # Obtener fechas para gráficos
    train_dates = train_df['Date'].iloc[SEQUENCE_LENGTH:].values
    val_dates = val_df['Date'].iloc[SEQUENCE_LENGTH:].values
    test_dates = test_df['Date'].iloc[SEQUENCE_LENGTH:].values
    
    # Graficar predicciones
    plot_predictions(train_preds, train_targets, ticker, "train", train_dates)
    plot_predictions(val_preds, val_targets, ticker, "validation", val_dates)
    plot_predictions(test_preds, test_targets, ticker, "test", test_dates)
    
    # Graficar curvas de entrenamiento
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(train_losses, label='Train Loss')
    ax.plot(val_losses, label='Val Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Weighted Loss')
    ax.set_title(f'{ticker} - Training Curves')
    ax.legend()
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    
    curve_path = os.path.join(PLOTS_PATH, f"{ticker}_training_curves.png")
    fig.savefig(curve_path, dpi=150, bbox_inches='tight')
    print(f"Saved training curves: {curve_path}")
    plt.close(fig)
    
    # Guardar métricas a CSV
    results = []
    for dataset_name, metrics in [('train', train_metrics_final), ('validation', val_metrics_final), ('test', test_metrics_final)]:
        for target_name, target_metrics in metrics.items():
            row = {
                'ticker': ticker,
                'dataset': dataset_name,
                'target': target_name,
                **target_metrics
            }
            results.append(row)
    
    results_df = pd.DataFrame(results)
    results_path = os.path.join(RESULTS_PATH, f"{ticker}_metrics.csv")
    results_df.to_csv(results_path, index=False)
    print(f"Saved metrics: {results_path}")
    
    # Guardar modelo
    model_path = os.path.join(RESULTS_PATH, f"{ticker}_model.pt")
    torch.save({
        'model_state_dict': model.state_dict(),
        'features': features,
        'feature_scaler': feature_scaler,
        'target_scalers': target_scalers,
        'target_weights': TARGET_WEIGHTS,
        'config': {
            'input_size': input_size,
            'hidden_size': HIDDEN_SIZE,
            'num_layers': NUM_LAYERS,
            'output_size': output_size,
            'dropout': DROPOUT,
            'architecture': 'StockLSTMMultiHead'
        }
    }, model_path)
    print(f"Saved model: {model_path}")
    
    # Extraer y guardar embeddings
    print("\nExtracting embeddings...")
    embeddings_path = '../data/embeddings/lstm_multihead/'
    os.makedirs(embeddings_path, exist_ok=True)
    
    train_embeddings, _ = extract_embeddings(model, train_loader, DEVICE)
    val_embeddings, _ = extract_embeddings(model, val_loader, DEVICE)
    test_embeddings, _ = extract_embeddings(model, test_loader, DEVICE)
    
    print(f"  Train embeddings: {train_embeddings.shape}")
    print(f"  Val embeddings: {val_embeddings.shape}") 
    print(f"  Test embeddings: {test_embeddings.shape}")
    
    # Guardar embeddings
    for split_name, embeddings, targets in [
        ('train', train_embeddings, train_targets),
        ('val', val_embeddings, val_targets),
        ('test', test_embeddings, test_targets)
    ]:
        emb_path = os.path.join(embeddings_path, f"{ticker}_{split_name}_embeddings.npz")
        np.savez_compressed(emb_path, embeddings=embeddings, targets=targets)
        print(f"Saved: {emb_path}")
    
    print(f"\n{ticker} processing complete!")
    
    return results_df

### Ejecutar Pipeline para Todos los Tickers

In [ ]:
print("\n" + "="*80)
print("LSTM Multi-Head Stock Embedding & Prediction Pipeline")
print("="*80)
print(f"Targets: {TARGETS}")
print(f"Target Weights: {TARGET_WEIGHTS}")
print(f"Train: {TRAIN_START}-{TRAIN_END}")
print(f"Val: {VAL_START}-{VAL_END}")
print(f"Test: {TEST_YEAR}")
print(f"Device: {DEVICE}")

all_results = []

for ticker in TICKERS:
    try:
        results = run_lstm_for_ticker(ticker)
        if results is not None:
            all_results.append(results)
    except Exception as e:
        print(f"\nError processing {ticker}: {e}")
        import traceback
        traceback.print_exc()
        continue

### Resultados Combinados

In [ ]:
# Combinar todos los resultados
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    combined_path = os.path.join(RESULTS_PATH, "all_tickers_metrics.csv")
    combined_results.to_csv(combined_path, index=False)
    print(f"\nCombined metrics saved: {combined_path}")
    
    # Estadísticas resumen
    print("\n" + "="*80)
    print("Summary Statistics (Test Set)")
    print("="*80)
    test_results = combined_results[combined_results['dataset'] == 'test']
    summary = test_results.groupby('target')[['MSE', 'RMSE', 'MAE', 'R2']].mean()
    display(summary)

print("\n" + "="*80)
print("Pipeline completed!")
print("="*80)